<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/translate_Laion_OIG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%pip install -q transformers datasets sentencePiece ftfy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.7 MB/s eta 0:00:00


In [7]:
import ftfy
import json
import torch
import spacy
import pickle
import warnings
import pandas as pd
from ftfy import fix_encoding
from tqdm.auto import tqdm
from spacy.lang.en import English
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer

pd.set_option('display.max_columns', 500)  
pd.set_option('display.expand_frame_repr', 500)
pd.set_option('max_colwidth', 100)
pd.set_option('display.max_rows', 500)  

warnings.filterwarnings("ignore")
# ---------------------------------------------------------------------------------------

if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu" 
print(dev, torch.cuda.get_device_name(0))
device = torch.device(dev)


# Model
model_name = 'Helsinki-NLP/opus-mt-en-ROMANCE'
marian_tokenizer = MarianTokenizer.from_pretrained(model_name)
marian_model = MarianMTModel.from_pretrained(model_name)

cuda:0 Tesla T4


# OIG English dataset

### details about the data, [here](https://laion.ai/blog/oig-dataset/)

In [9]:
data_en = load_dataset('0-hero/OIG-small-chip2')
data_en

  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['user', 'chip2'],
        num_rows: 210289
    })
})

In [10]:
data_en['train']['user'][-1], data_en['train']['chip2'][-1]

('Matt rode his bike away from the house, he rode a total of 1000 feet. Along the way, he passed a stop sign that was 350 feet away from his house. He continued his ride and passed a second sign. After passing the second sign he road an additional 275 feet. How many feet are between the first and second signs?\n\n',
 'How many feet are between the first and second signs? ** Matthew rode 350 feet to the first sign, so there are 1000-350 =<<1000-350=650>>650 feet between the first sign and the end.\nHow many feet are between the first and second signs? ** After passing the second sign Matthew road 275, so 650-275 = <<650-275=375>>375 feet remain.\n#### 375')

In [70]:
nlp = English()
nlp.add_pipe('sentencizer')
def chunkstring_spacy(text):
    """
    Segment text and prepare to translation

    Args:
      text: Sentence to be translated
      
    Returns:
      Segmented text.
    """
    chunck_sentences = []
    doc = nlp(str(text.replace('\n', '')))
    for sent in doc.sents:
        chunck_sentences.append('>>pt_br<<' + ' ' + sent.text)
        
    return chunck_sentences
    
def translate(aux_sent):
    """
    Translate text

    Args:
      aux_sent: Sentence to be translated
      
    Returns:
      Translated text.
    """
    max_length = 512
    num_beams = 1

    sentence = chunkstring_spacy(aux_sent)

    #Move o modelo para a GPU
    marian_model.to(device)
    marian_model.eval()

    tokenized_text = marian_tokenizer.prepare_seq2seq_batch(sentence, max_length=max_length, return_tensors='pt')
                        
    translated = marian_model.generate(input_ids=tokenized_text['input_ids'].to(device), 
                                        max_length=max_length, 
                                        num_beams=num_beams, 
                                        early_stopping=True, 
                                        do_sample=False)
                        
    tgt_text = [fix_encoding(marian_tokenizer.decode(t, skip_special_tokens=True)) for t in translated]
    return ' '.join(tgt_text)

In [ ]:
def pickle_file(path, data=None):
    if data is None:
        with open(path, 'rb') as f:
            return pickle.load(f)
    if data is not None:
        with open(path, 'wb') as handle:
            pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

path_save = '/content/drive/MyDrive/LLMs/data/OIG/'

user, chip2 = [], []

SAVE_EVERY = 5_000 # +- 44 checkpoints in 220k samples

loop = tqdm(data_en['train'], leave=True)
for ix, inputs in enumerate(loop, 1):
    user_ptbr = translate(inputs['user'])
    chip_ptbr = translate(inputs['chip2'])
    user.append(user_ptbr)
    chip2.append(chip_ptbr)
    
    
    if len(user) == len(chip2) and len(user) == SAVE_EVERY: 
        print(f"\n\n{ix}: {inputs['user']}")
        print(f"{ix}: {chip_ptbr}\n")
        pickle_file(path=path_save+f'user_{ix}', data=user)
        pickle_file(path=path_save+f'chip2_{ix}', data=chip2)
        user, chip2 = [], []

    loop.set_description(f"examples processed: {ix}")

  0%|          | 0/210289 [00:00<?, ?it/s]

In [ ]:
import glob

all_user, all_chip2 =[],[]
for user_file, chip2_file in zip(
    sorted(glob.glob(path_save+'user*')), 
    sorted(glob.glob(path_save+'chip2*'))):
    
    if user_file[-3:] == chip2_file[-3:]:
        all_user += pickle_file(user_file)
        all_chip2 += pickle_file(chip2_file)

assert len(all_user) == len(all_chip2), "load pickle FAILED"

all_user[0], all_chip2[0]

In [ ]:
data_en['train']['user'][0], data_en['train']['chip2'][0]